# logging

[logging](https://docs.python.org/3/library/logging.html) 是用于打印日志的 Python 内置库。

logging 和普通的 print 函数的区别在于，使用 logging 一般是为了报告代码运行的状态，而非输出结果。logging 提供了一系列功能，使打印日志更加方便快捷，比如使用 formatter 可以轻松定制日志的打印格式。

本文目录：

1. 简单使用
2. 在单个脚本中使用
    - 威胁等级 - level
    - 日志打印格式 - format参数
3. 涉及多个模块的使用
    - 无法被重写
    - Logger

logging 有五个函数 debug(), info(), warning(), error() 和 critical()，分别代表五种信息等级：

- **DEBUG**: Detailed information, typically of interest only when diagnosing problems.
- **INFO**: Confirmation that things are working as expected.
- **WARNING**: An indication that something unexpected happened, or indicative of some problem in the near future (e.g. ‘disk space low’). The software is still working as expected.
- **ERROR**: Due to a more serious problem, the software has not been able to perform some function.
- **CRITICAL**: A serious error, indicating that the program itself may be unable to continue running.

## 1. 简单使用

最简单的使用 logging 的方式是:

In [1]:
import logging

# 会在命令行中输出
logging.warning('Watch out!')
logging.error('It does not work!')
logging.critical('Wow!')

ERROR:root:It does not work!
CRITICAL:root:Wow!


In [2]:
import logging

# 不会在命令行中输出
logging.info('OK, it is fine')
logging.debug('Relax, bro')

像上面这样直接调用方法，就是最简单的使用方式了。非常简单易用对吧，下面我们再深入一些。

## 2. 在单个脚本中使用

logging 在单个脚本中使用时，我们可以在 import logging 之后，对其做一个统一配置。

### 2.1 威胁等级 - level

In [3]:
import logging


logging.basicConfig(filename='example.log', level=logging.DEBUG)

logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')
logging.error('And non-ASCII stuff, too, like Øresund and Malmö')
logging.critical('Can you see me?')

ERROR:root:And non-ASCII stuff, too, like Øresund and Malmö
CRITICAL:root:Can you see me?


新建 Python 文件，将上述的代码复制到文件中执行。

它会把全部四条信息：

```
DEBUG:root:This message should go to the log file
INFO:root:So should this
WARNING:root:And this, too
ERROR:root:And non-ASCII stuff, too, like Øresund and Malmö
```

打印到名为 `example.log` 的日志文件中。

这是因为我们的 level 被设为 debug。信息所代表的威胁等级，debug < info < warning < error < critical。

level 关键字参数代表的是一个阈值，所有大于等于 level 所代表的威胁等级的信息，都将被打印。

同样的代码，如果我们将 level 修改为 logging.ERROR，则打印到日志文件中的信息就只剩 error 和 critical 两个等级的信息了。

### 2.2 日志打印格式 - format

In [4]:
import logging

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.DEBUG)
logging.warning('Hi there.')

看起来好像在Jupyter Notebook 似乎不起作用，没办法呈现效果。可以把它粘贴到脚本中运行试试～

> 关于 format 选项，可查看：[LogRecord attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes) 

## 3. 涉及多个模块的使用


在多个模块中使用 logging 时，有一些特殊的技巧。

### 3.1 无法被重写

由于 logging 无法被重写，因此如果涉及到多个模块，直接使用 `logging.warning()` 的用法可能会导致程序出现用户未预期的表现。

In [5]:
import logging

# 第一次定义
logging.basicConfig(level=logging.ERROR)

# 重写定义（不起效果）
logging.basicConfig(level=logging.DEBUG)

logging.debug('This message should go to the log file')
logging.info('So should this')
logging.warning('And this, too')
logging.error('And non-ASCII stuff, too, like Øresund and Malmö')
logging.critical('Can you see me?')

ERROR:root:And non-ASCII stuff, too, like Øresund and Malmö
CRITICAL:root:Can you see me?


将上述代码粘贴到脚本执行吧，看看对 level 的重写是否改变了 logging 的打印方式。

**解决方案**：

如果你需要在一个脚本中调用多个模块，并且在那些被调用的模块中也使用 logging，但是又不想本脚本的 logging 的行为被 import 的包中的 logging 的定义影响，那么你就需要另一种“写法”。

In [6]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logger.warning('Hi there!')

通过指定不同模块的不同 logger，我们可以确保模块见的 logging 配置不会相互影响！

### 3.2 Logger

Logger 的成员方法包括：



- `Logger.setLevel()` specifies the lowest-severity log message a logger will handle, where debug is the lowest built-in severity level and critical is the highest built-in severity. For example, if the severity level is INFO, the logger will handle only INFO, WARNING, ERROR, and CRITICAL messages and will ignore DEBUG messages.

- `Logger.addHandler()` and `Logger.removeHandler()` add and remove handler objects from the logger object. Handlers are covered in more detail in Handlers.

- `Logger.addFilter()` and `Logger.removeFilter()` add and remove filter objects from the logger object. Filters are covered in more detail in Filter Objects.

> 关于 logger 及其成员方法，可参考：[Loggers](https://docs.python.org/3/howto/logging.html#loggers)

有两种 Handler 比较常用，分别是：**StreamHandler** 和 **FileHandler**。StreamHandler 控制命令行输出，FileHandler 控制日志文件输出。

先看看 StreamHandler。

In [7]:
# StreamHandler
import logging

# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
stream_handler = logging.StreamHandler()
stream_handler.setLevel(logging.ERROR)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)  # add formatter to stream_handler

# add stream_handler to logger
logger.addHandler(stream_handler)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

DEBUG:simple_example:debug message
INFO:simple_example:info message
2020-10-17 23:55:54,701 - simple_example - ERROR - error message
ERROR:simple_example:error message
2020-10-17 23:55:54,702 - simple_example - CRITICAL - critical message
CRITICAL:simple_example:critical message


看上述代码会发现有两行代码都设置了 level:

```python
# create logger
logger.setLevel(logging.DEBUG)

# create console handler
stream_handler.setLevel(logging.ERROR)
```

一次在创建 logger 的时候，一次在设置 Handler 的时候。

两次设置对配置结果影响的表现是：如果 Handler 的 level 比 logger 的 level 高，则按 Handler 的 level 输出；否则按 logger 的 level 输出。

其本质是：日志信息流过两级过滤器，先流经 logger.level，后流经 stream_handler.level。通过了两级过滤器后的信息将被输出。

In [8]:
print(logger.level)
print(stream_handler.level)

10
40


再看 FileHandler。

In [9]:
# FileHandler
import logging

# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.INFO)

# create file handler and set level to debug
file_handler = logging.FileHandler('error.log')
file_handler.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)  # add formatter to file_handler

# add file_handler to logger
logger.addHandler(file_handler)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

INFO:simple_example:info message
2020-10-17 23:55:54,716 - simple_example - ERROR - error message
ERROR:simple_example:error message
2020-10-17 23:55:54,717 - simple_example - CRITICAL - critical message
CRITICAL:simple_example:critical message


相较于 StreamHandler，FileHandler 在创建时需要传入一个 string 类型变量作为日志文件名。

参考：

1. [Logging HOWTO](https://docs.python.org/3/howto/logging.html#logging-howto)
2. [Python Tutorial: Logging Basics - Logging to Files, Setting Levels, and Formatting](https://youtu.be/-ARI4Cz-awo)
3. [Python Tutorial: Logging Advanced - Loggers, Handlers, and Formatters](https://youtu.be/jxmzY9soFXg)